In [47]:
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler, LabelEncoder


In [23]:
dataset_dir = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(''))), 'datasets')

##### Datos a usar

- V2 --> dataset limpio
- V4 --> variables construidas a partir de otras + agrupacion de violencias absorbidas
- V5 --> eliminación de las variables cn poca información

##### Modificaiones para SVM

1. pasar datetime a timestamp y escalar --> OK
2. escalar la variable edad --> OK
3. pasar edad a categórica (en una columna distinta) --> OK
4. pasar todas las categóricas a dummy:
   1. edad, momento del día y estación del año con un encoder ordinal
   2. el resto con un encoder que no codifique ordinalidad
5. borrar las categóricas para hacer más pequeño el dataset
6. guardar los datasets como sus versiones en SVM

In [41]:
llamados_v2= pd.read_excel(os.path.join(dataset_dir, 'xlsx/llamados_v2.xlsx'), parse_dates=['llamado_fecha_hora'])
llamados_v4= pd.read_excel(os.path.join(dataset_dir, 'xlsx/llamados_v4.xlsx'), parse_dates=['llamado_fecha_hora'])
llamados_v5= pd.read_excel(os.path.join(dataset_dir, 'xlsx/llamados_v5.xlsx'), parse_dates=['llamado_fecha_hora'])


In [42]:
datasets_svm = [llamados_v2, llamados_v4, llamados_v5]
datasets_45 = [llamados_v4, llamados_v5]

In [ ]:
scaler = MinMaxScaler()
encoder = OrdinalEncoder()
label_encoder = LabelEncoder()

In [43]:
# 1. fecha hora a timestamp y escalado

for dataset in datasets_svm:
    dataset['timestamp_encoded'] = encoder.fit_transform(dataset[['llamado_fecha_hora']])
    dataset['timestamp_scaled'] = scaler.fit_transform(dataset[['timestamp_encoded']])
    dataset.drop('timestamp_encoded', axis=1, inplace=True)
    dataset.drop('llamado_fecha_hora', axis=1, inplace=True )

In [44]:
# 2. escalar edades
for dataset in datasets_svm:
    dataset['victima_edad_escalada'] = scaler.fit_transform(dataset[['victima_edad']])
    dataset['llamante_edad_escalada'] = scaler.fit_transform(dataset[['llamante_edad']])

In [45]:
# 3. pasar edad a categórica


def categoria_edad (x):
    if (x >= 0) and (x <= 11) :
        return 'Niñez'
    elif (x >= 12) and (x <=18):
        return 'Adolescencia'
    elif (x >= 19) and (x <=30):
        return 'Juventud'
    elif (x>=31) and (x<=65) :
        return 'Vejez'
    elif x>=66:
        return 'Vejez mayor'
    else:
        return 'NS/NC'

for dataset in datasets_svm:
    dataset['victima_edad_cat'] = \
        dataset.victima_edad.apply(categoria_edad)
    dataset['llamante_edad_cat'] = \
        dataset.llamante_edad.apply(categoria_edad)
    dataset.drop(['victima_edad','llamante_edad'], axis=1, inplace=True)




In [53]:
# 4.1 pasar edad_cat, momento_dia (V4, V5) y estacion_del_año (V4, V5) a dummy con un encoder ordinal


for dataset in datasets_svm:
    dataset['victima_edad_cat_dummy'] = label_encoder.fit_transform(dataset['victima_edad_cat'])
    dataset['llamante_edad_cat_dummy'] = label_encoder.fit_transform(dataset['llamante_edad_cat'])
    dataset.drop(['victima_edad_cat', 'llamante_edad_cat'], axis=1, inplace=True)

for dataset in datasets_45:
    dataset['momento_dummy'] = label_encoder.fit_transform(dataset['momento_dia'])
    dataset['estacion_dummy'] = label_encoder.fit_transform(dataset['estacion_del_año'])
    dataset.drop(['estacion_del_año','momento_dia',], axis=1, inplace=True)


In [ ]:
# escalar las variables ordinales edad, momento del día y estación

for dataset in datasets_svm:
    dataset['victima_edad_dummy_scaled'] = scaler.fit_transform(dataset[['victima_edad_cat_dummy']])
    dataset['llamante_edad_dummy_scaled'] = scaler.fit_transform(dataset[['llamante_edad_cat_dummy']])
    dataset.drop('victima_edad_cat_dummy', axis=1, inplace=True)
    dataset.drop('llamante_edad_cat_dummy', axis=1, inplace=True )

for dataset in datasets_45:
    dataset['momento_dummy_scaled'] = scaler.fit_transform(dataset[['momento_dummy']])
    dataset['estacion_dummy_scaled'] = scaler.fit_transform(dataset[['estacion_dummy']])
    dataset.drop('momento_dummy', axis=1, inplace=True)
    dataset.drop('estacion_dummy', axis=1, inplace=True )


In [76]:
# 4.2 pasar el resto de las cat a dummy con un encoder no ordinal

# pasar todos los objects en la lista datasets_svm a one hot encoding y ahí veo qué tan grande queda el dataset
# son 34 variables object

llamados_v2.llamado_provincia.dtype == 'object'



True